<a href="https://colab.research.google.com/github/Aryansalve/VGG16-Classifier/blob/main/VGG_Classifier_pre_trainedipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#The python file and the archive have the same location
from zipfile import ZipFile
file_name = "/content/gdrive/MyDrive/Classifier project/archive.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np
import os
from random import shuffle
import shutil

main_path='./output/bee-vs-wasp'
training_path=main_path+'/training'
testing_path=main_path+'/validation'
bee_training=training_path+'/bee'
wasp_training=training_path+'/wasp'
other_insects_training=training_path+'/other_insects'
other_noinsects_training=training_path+'/other_noinsects'
bee_testing=testing_path+'/bee'
wasp_testing=testing_path+'/wasp'
other_insects_testing=testing_path+'/other_insects'
other_noinsects_testing=testing_path+'/other_noinsects'
def directory_creation(path):
    os.mkdir(path)
directory_creation('./output')
directory_creation(main_path)
directory_creation(training_path)
directory_creation(testing_path)
directory_creation(bee_training)
directory_creation(wasp_training)
directory_creation(other_insects_training)
directory_creation(other_noinsects_training)
directory_creation(bee_testing)
directory_creation(wasp_testing)
directory_creation(other_insects_testing)
directory_creation(other_noinsects_testing)

In [ ]:
def copy_files(original,dest_training,dest_testing):
    l=os.listdir(original)
    training_length=int(len(l)*0.8)
    shuffle(l)
    for i in range(len(l)):
        if i<training_length:
            shutil.copy(original+'/'+l[i],dest_training)
        else:
            shutil.copy(original+'/'+l[i],dest_testing)

In [ ]:
copy_files('/content/kaggle_bee_vs_wasp/bee1',bee_training,bee_testing)
copy_files('/content/kaggle_bee_vs_wasp/bee2',bee_training,bee_testing)
copy_files('/content/kaggle_bee_vs_wasp/wasp1',wasp_training,wasp_testing)
copy_files('/content/kaggle_bee_vs_wasp/wasp2',wasp_training,wasp_testing)
copy_files('/content/kaggle_bee_vs_wasp/other_insect',other_insects_training,other_insects_testing)
copy_files('/content/kaggle_bee_vs_wasp/other_noinsect',other_noinsects_training,other_noinsects_testing)

In [ ]:
base_dir = '/content/output/bee-vs-wasp'
train_dir = '/content/output/bee-vs-wasp/training'
train_bee_dir = '/content/output/bee-vs-wasp/training/bee'
train_wasp_dir = '/content/output/bee-vs-wasp/training/wasp'
train_insect_dir = '/content/output/bee-vs-wasp/training/other_insects'
train_other_dir = '/content/output/bee-vs-wasp/training/other_noinsects'
test_dir = '/content/output/bee-vs-wasp/validation'
test_bee_dir = '/content/output/bee-vs-wasp/validation/bee'
test_wasp_dir = '/content/output/bee-vs-wasp/validation/wasp'
test_insect_dir = '/content/output/bee-vs-wasp/validation/other_insects'
test_other_dir = '/content/output/bee-vs-wasp/validation/other_noinsects'

# valid_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation'
# valid_bee_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/bee'
# valid_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/wasp'

In [ ]:
num_bee_train = len(os.listdir(train_bee_dir))
num_wasp_train = len(os.listdir(train_wasp_dir))
num_insect_train = len(os.listdir(train_insect_dir))
num_other_train = len(os.listdir(train_other_dir))
num_bee_test = len(os.listdir(test_bee_dir))
num_wasp_test= len(os.listdir(test_wasp_dir))
num_insect_test = len(os.listdir(test_insect_dir))
num_other_test= len(os.listdir(test_other_dir))

# num_bee_validaition = len(os.listdir(valid_bee_dir))
# num_wasp_validation= len(os.listdir(valid_malign_dir))

In [ ]:
print("Total Training bee Images",num_bee_train)
print("Total Training wasp Images",num_wasp_train)
print("Total Training other Images",num_other_train)
print("Total Training insect Images",num_insect_train)
print("--")
print("Total Test bee Images", num_bee_test)
print("Total Test wasp Images",num_wasp_test)
print("Total Test other Images", num_other_test)
print("Total Test insect Images",num_insect_test)
total_train = num_bee_train+num_wasp_train+num_other_train+num_insect_train
total_test = num_bee_test+num_wasp_test+num_other_test+num_insect_test
print("--")
print("Total Training Images",total_train)
print("Total Testing Images",total_test)

Total Training bee Images 2546
Total Training wasp Images 3953
Total Training other Images 684
Total Training insect Images 1951
--
Total Test bee Images 637
Total Test wasp Images 990
Total Test other Images 172
Total Test insect Images 488
--
Total Training Images 9134
Total Testing Images 2287


In [ ]:
IMG_SHAPE  = 224
batch_size = 32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 9134 images belonging to 4 classes.
Found 2287 images belonging to 4 classes.


In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
checkpoint_path = '/content/gdrive/MyDrive/Classifier project/vgg16_model_bee'

In [ ]:
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             save_best_only=False,
                             mode='min', verbose=1)

In [ ]:
# Initial run
# vgg_classifier = model.fit(train_data_gen,
# steps_per_epoch=(total_train//(2*batch_size)),
# epochs = 10,
# validation_data=test_data_gen,
# validation_steps=(total_test//(2*batch_size)),
# batch_size = batch_size,callbacks=[checkpoint])

Epoch 1/10
142/142 [==============================] - ETA: 0s - loss: 1.2091 - acc: 0.5285 
Epoch 1: saving model to /content/drive/MyDrive/vgg16_model_bee


142/142 [==============================] - 3312s 23s/step - loss: 1.2091 - acc: 0.5285 - val_loss: 0.7064 - val_acc: 0.7536
Epoch 2/10
142/142 [==============================] - ETA: 0s - loss: 0.9030 - acc: 0.6425 
Epoch 2: saving model to /content/drive/MyDrive/vgg16_model_bee


142/142 [==============================] - 3152s 22s/step - loss: 0.9030 - acc: 0.6425 - val_loss: 0.6484 - val_acc: 0.7571
Epoch 3/10
142/142 [==============================] - ETA: 0s - loss: 0.7840 - acc: 0.6912 
Epoch 3: saving model to /content/drive/MyDrive/vgg16_model_bee


142/142 [==============================] - 3105s 22s/step - loss: 0.7840 - acc: 0.6912 - val_loss: 0.5806 - val_acc: 0.7893
Epoch 4/10
142/142 [==============================] - ETA: 0s - loss: 0.7313 - acc: 0.7209 
Epoch 4: saving model to /content/drive/MyDrive/vgg16_model_bee


142/142 [==============================] - 3103s 22s/step - loss: 0.7313 - acc: 0.7209 - val_loss: 0.5332 - val_acc: 0.8045
Epoch 5/10
142/142 [==============================] - ETA: 0s - loss: 0.6990 - acc: 0.7375 
Epoch 5: saving model to /content/drive/MyDrive/vgg16_model_bee


142/142 [==============================] - 3103s 22s/step - loss: 0.6990 - acc: 0.7375 - val_loss: 0.5279 - val_acc: 0.7991
Epoch 6/10
  8/142 [>.............................] - ETA: 39:03 - loss: 0.6935 - acc: 0.7031

KeyboardInterrupt: ignored

In [ ]:
# Only run when resuming !!
model.load_weights("/content/gdrive/MyDrive/Classifier project/vgg16_model_bee")


In [ ]:
#Resumed Run
model.fit(train_data_gen,
          batch_size=batch_size,
          steps_per_epoch=(total_train//(2*batch_size)),
          initial_epoch=5,
          epochs=10,
          validation_data=test_data_gen,
          validation_steps=(total_test//(2*batch_size)),
          callbacks=[checkpoint])

# model.fit(train_data_gen,
#           steps_per_epoch=(total_train//(2*batch_size)),
#           epochs=10, # Set the total number of epochs you want to train
#           initial_epoch=5, # Set the last completed epoch (last epoch + 1) from the previous training
#           batch_size=batch_size,
#           validation_data=test_data_gen,
#           validation_steps=(total_test//(2*batch_size)),
#           callbacks=[checkpoint])

Epoch 6/10
142/142 [==============================] - ETA: 0s - loss: 0.6622 - acc: 0.7459 
Epoch 6: saving model to /content/gdrive/MyDrive/Classifier project/vgg16_model_bee


142/142 [==============================] - 3208s 23s/step - loss: 0.6622 - acc: 0.7459 - val_loss: 0.5251 - val_acc: 0.8196
Epoch 7/10
142/142 [==============================] - ETA: 0s - loss: 0.6476 - acc: 0.7464 
Epoch 7: saving model to /content/gdrive/MyDrive/Classifier project/vgg16_model_bee


142/142 [==============================] - 3216s 23s/step - loss: 0.6476 - acc: 0.7464 - val_loss: 0.5314 - val_acc: 0.7973
Epoch 8/10
142/142 [==============================] - ETA: 0s - loss: 0.6495 - acc: 0.7424 

In [ ]:
#Resumed Run
model.fit(train_data_gen,
          batch_size=batch_size,
          steps_per_epoch=(total_train//(2*batch_size)),
          initial_epoch=7,
          epochs=10,
          validation_data=test_data_gen,
          validation_steps=(total_test//(2*batch_size)),
          callbacks=[checkpoint])

Epoch 8/10
142/142 [==============================] - ETA: 0s - loss: 0.6458 - acc: 0.7456 
Epoch 8: saving model to /content/gdrive/MyDrive/Classifier project/vgg16_model_bee


142/142 [==============================] - 3780s 27s/step - loss: 0.6458 - acc: 0.7456 - val_loss: 0.4785 - val_acc: 0.8295
Epoch 9/10
142/142 [==============================] - ETA: 0s - loss: 0.6451 - acc: 0.7499 
Epoch 9: saving model to /content/gdrive/MyDrive/Classifier project/vgg16_model_bee


142/142 [==============================] - 3623s 26s/step - loss: 0.6451 - acc: 0.7499 - val_loss: 0.4909 - val_acc: 0.8357
Epoch 10/10
 59/142 [===========>..................] - ETA: 28:07 - loss: 0.6500 - acc: 0.7463